In [1]:
%load_ext autoreload
%autoreload 2
#!/usr/bin/env python

try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

import math
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import sys

sys.path.insert(0, "../../scripts")
import os
import numpy as np
from astropy.table import Table
from numpy import random
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData
from clmm import Cosmology
from clmm.support import mock_data as mock
import pandas as pd
import numpy as np
import pyccl as ccl
from firecrown.sacc_support import sacc

clmm.__version__
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import scipy.integrate
import astropy.units as u
import itertools

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

%matplotlib inline


In [2]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.table import Table, QTable, hstack, vstack
from astropy import units as u
import corner
cosmo_astropy = FlatLambdaCDM(H0=71.0, Om0=0.265, Ob0 = 0.0448)
import iminuit
from iminuit import Minuit
cosmo_astropy.critical_density(0.4).to(u.Msun / u.Mpc**3).value


sys.path.append('/pbs/throng/lsst/users/cpayerne/CLMassDC2/modules/')

import analysis_Mass_Richness_relation as analysis
import multiprocessing
import clmm
import clmm.dataops
from clmm.dataops import compute_tangential_and_cross_components, make_radial_profile, make_bins
from clmm.galaxycluster import GalaxyCluster
import clmm.utils as u
import astropy.units as un
from clmm import Cosmology
from clmm.support import mock_data as mock
import pyccl as ccl

cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_astropy = FlatLambdaCDM(H0=71.0, Om0=0.265, Ob0 = 0.0448)
cosmo_clmm = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_ccl  = ccl.Cosmology(Omega_c=0.265-0.0448, Omega_b=0.0448, h=0.71, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)
area = 439.78987

In [3]:
sys.path.insert(0, "/pbs/home/e/ebarroso/gcr-catalogs")
import GCRCatalogs

GCRCatalogs.set_root_dir_by_site("in2p3")

import matplotlib.pyplot as plt
import pickle
import numpy as np
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.table import Table
def load(filename, **kwargs):
    """Loads GalaxyCluster object to filename using Pickle"""
    with open(filename, 'rb') as fin:
        return pickle.load(fin, **kwargs)

## Extract Catalog

In [4]:
name = '/pbs/throng/lsst/users/cpayerne/CLMassDC2/data/matched_pairs_Mfofcut.fits'
dat = fits.open(name)

In [5]:
dat_open= dat[1].data

In [6]:
richness = dat_open['cat1_richness']
redshift = dat_open['cat2_z']
M200c = dat_open['cat2_M200c']

In [7]:
table = Table()

In [8]:
table['richness'] = dat_open['cat1_richness']
table['M200c'] = dat_open['cat2_M200c']
table['redshift'] = dat_open['cat1_ztrue_cg']


In [9]:
table = table[(table['M200c'] > 1e13)*(table['richness'] > 20)*(table['redshift'] < .8)*(table['redshift'] > .2)]
area = 439.78987

In [10]:
print(table)

 richness        M200c         redshift 
---------- ------------------ ----------
 190.70665  326491145207346.5 0.51072353
 179.12813  645749230889104.2  0.5740297
 167.33603  216337687012597.2  0.6426174
  111.9147  322972230485814.1 0.43919736
104.439766  278007164916462.0  0.4680865
 103.49396  393139675482140.9 0.44349608
  89.59668 439309794128045.06  0.5693116
  77.40795  299131074023481.7  0.4968995
  82.35056 449733785617307.06 0.49747086
  71.94667  435095924586005.6  0.4326079
       ...                ...        ...
  22.38326  176028960519932.4  0.4172357
  25.81563 166031807398580.28 0.58921903
  26.09022 112657280458636.62 0.78044695
 20.979803 103729863896338.03 0.20682013
 20.552664 208625744304676.06  0.5949888
 24.808802  36297902616518.31  0.5031447
 23.649424  78001713694287.33 0.64465666
 20.598429 22310743544788.734 0.65758795
 22.047651 44988473920901.414 0.70174426
  20.17129  82455013012552.11  0.6335203
 24.061308 218560431833780.28  0.6059585
Length = 2669 ro

## BINN DATA

In [11]:
data_table = table
cluster_z = data_table["redshift"]
cluster_richness = np.log10(data_table["richness"])
cluster_logM = np.log10(data_table["M200c"])

N_richness = 4  # number of richness bins
N_z = 3  # number of redshift bins

cluster_counts, z_edges, richness_edges, _ = stats.binned_statistic_2d(
    cluster_z, cluster_richness, cluster_logM, "count", bins=[N_z, N_richness]
)

mean_logM = stats.binned_statistic_2d(
    cluster_z,
    cluster_richness,
    cluster_logM,
    "mean",
    bins=[z_edges, richness_edges],
).statistic

std_logM = stats.binned_statistic_2d(
    cluster_z, cluster_richness, cluster_logM, "std", bins=[z_edges, richness_edges]
).statistic

var_mean_logM = std_logM**2 / cluster_counts 

covariance = np.diag(
    np.concatenate((cluster_counts.flatten(), var_mean_logM.flatten()))
)

## SAVE TO SACC

In [12]:
s_count = sacc.Sacc()
bin_z_labels = []
bin_richness_labels = []

survey_name = "redmapper"
s_count.add_tracer("cluster_survey", survey_name, area)

for i, z_bin in enumerate(zip(z_edges[:-1], z_edges[1:])):
    lower, upper = z_bin
    bin_z_label = f"bin_z_{i}"
    s_count.add_tracer("bin_z", bin_z_label, lower, upper)
    bin_z_labels.append(bin_z_label)

for i, richness_bin in enumerate(zip(richness_edges[:-1], richness_edges[1:])):
    lower, upper = richness_bin
    bin_richness_label = f"bin_rich_{i}"
    s_count.add_tracer(
        "bin_richness", bin_richness_label, lower, upper
    )
    bin_richness_labels.append(bin_richness_label)

cluster_count = sacc.standard_types.cluster_counts
cluster_mass = sacc.standard_types.cluster_mean_log_mass

counts_and_edges = zip(
    cluster_counts.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
)

# mean_logM_and_edges = zip(
#     mean_logM.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
# )

mean_logM_and_edges = zip(
    mean_logM.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
)


for counts, (bin_z_label, bin_richness_label) in counts_and_edges:
    s_count.add_data_point(
        cluster_count, (survey_name, bin_z_label, bin_richness_label), int(counts)
    )
    
for bin_mean_logM, (bin_z_label, bin_richness_label) in mean_logM_and_edges:
    s_count.add_data_point(
        cluster_mass,
        (survey_name, bin_z_label, bin_richness_label),
        bin_mean_logM,
    )

# Then the add the covariance and save the file
s_count.add_covariance(covariance)
s_count.to_canonical_order()
s_count.save_fits("redmapper_full_sacc_data.fits", overwrite=True)